In [1]:
import os
rootDir = 'D:\\RJ\\UCI\\Ralph School\\2023 Spring\\CS 121\\Assignments\\.vscode\\Res\\DEV'
rootDir += '\\aiclub_ics_uci_edu'

jsonFiles = []
for root, dirs, files in os.walk(rootDir):
    for name in files:
        if name.endswith((".json")):
            full_path = os.path.join(root, name)
            jsonFiles.append(full_path)

In [2]:
for file in jsonFiles:
    print(file)

D:\RJ\UCI\Ralph School\2023 Spring\CS 121\Assignments\.vscode\Res\DEV\aiclub_ics_uci_edu\8ef6d99d9f9264fc84514cdd2e680d35843785310331e1db4bbd06dd2b8eda9b.json
D:\RJ\UCI\Ralph School\2023 Spring\CS 121\Assignments\.vscode\Res\DEV\aiclub_ics_uci_edu\906c24a2203dd5d6cce210c733c48b336ef58293212218808cf8fb88edcecc3b.json
D:\RJ\UCI\Ralph School\2023 Spring\CS 121\Assignments\.vscode\Res\DEV\aiclub_ics_uci_edu\9a59f63e6facdc3e5fe5aa105c603b545d4145769a107b4dc388312a85cf76d5.json


In [3]:
import json
from HelperClass import InvertedIndex, Posting, Token
from helper import tokenizeHtml

In [4]:
# Opening JSON file
jFile = open('Res.json')
data = json.load(jFile)
data['encoding']
jFile.close()

In [5]:
text_file = open("simple.html", "r", encoding='utf-8')
htmlContent = text_file.read()
text_file.close()

In [6]:
invIndex = InvertedIndex() #Create inverted index to hold tokens from parser

#Cleans and parses HTML content into tokens then adds it to Inverted index
tokenizeHtml(docId=1, invIndex=invIndex, htmlContent=htmlContent)

In [7]:
print(invIndex)

Total Tokens: 13
Token: my, Total Freq: 2
	DocId: 1, Freq: 2
		Positions: {1, 5}
		Fields:
			title:{1}
			h2:{5}

Token: page, Total Freq: 2
	DocId: 1, Freq: 2
		Positions: {2, 6}
		Fields:
			title:{2}
			a:{6}
			h2:{6}

Token: welcom, Total Freq: 1
	DocId: 1, Freq: 1
		Positions: {3}
		Fields:
			h2:{3}

Token: to, Total Freq: 1
	DocId: 1, Freq: 1
		Positions: {4}
		Fields:
			h2:{4}

Token: thi, Total Freq: 1
	DocId: 1, Freq: 1
		Positions: {7}
		Fields:
			normal:{7}

Token: is, Total Freq: 1
	DocId: 1, Freq: 1
		Positions: {8}
		Fields:
			normal:{8}

Token: the, Total Freq: 2
	DocId: 1, Freq: 2
		Positions: {9, 15}
		Fields:
			normal:{9, 15}

Token: first, Total Freq: 1
	DocId: 1, Freq: 1
		Positions: {10}
		Fields:
			normal:{10}

Token: paragraph, Total Freq: 1
	DocId: 1, Freq: 1
		Positions: {11}
		Fields:
			normal:{11}

Token: pride, Total Freq: 1
	DocId: 1, Freq: 1
		Positions: {12}
		Fields:
			normal:{12}

Token: cometh, Total Freq: 1
	DocId: 1, Freq: 1
		Positions: {1